In [1]:
import os
import re
from docx import Document
from docx.shared import Cm, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [2]:
def excitation(f):
    regulator = f[:4].upper()
    mode = ""
    err = 0
    if f.find("1нв") > -1:
        mode = "начального возбуждения "
    elif f.find("2гаш") > -1:
        mode = "гашения поля "
    t = ""                    
    if f.find("авт") > -1:
        t = "в автоматическом режиме"
    elif f.find("руч") > -1:
        t = "в ручном режиме"
    elif f.find("гщу") > -1:
        t = "в автоматическом режиме с ГЩУ"
    elif f.find("бщу") > -1:
        t = "в автоматическом режиме с БЩУ"
    elif f.find("птк") > -1:
        t = "в автоматическом режиме с ПТК"
    elif f.find("агп") > -1:
        t = "отключением АГП"
    elif f.find("кнв") > -1:
        t = "с КНВ"
    elif f.find("неусп") > -1:
        t = "неуспешного "
        err = 1
    if err == 1:
        return regulator + ", опыт "  + t + mode
    return regulator + ", опыт " + mode + t

def transfers(f):
    regulator = f[:4].upper()
    if f[4:-4].find(" пер ") > -1:
        str_out = f[4:-4].replace(" пер ", ", опыт перевода ")
    else:
        str_out = ", опыт перевода" + f[4:-4]
    return regulator + str_out.replace("авт", "автоматической режим").replace("руч", "ручной режим").replace("усин", " напряжения синхронизации").replace("3х100", "напряжения генератора").replace("пит", " питания").replace(" откл ", " потеря ")

def mode(f):
    regulator = f[:4].upper()
    result = regulator + ", "
    if f.find("vhz") > -1:
        result += "проверка вольт-герцового ограничителя"
    elif f.find("омтр") > -1:
        match = [""]
        if re.search(r'632=\d.\d+', f):
            match = re.search(r'632=\d.\d+', f)
        result += "проверка ОМТР с заниженной уставкой SetLimIfMaxOff " + match[0]
    elif f.find("осин") > -1:
        match = [""]
        if re.search(r'690=\d.\d+', f):
            match = re.search(r'690=\d.\d+', f)
        result += "проверка ОНС в ручном режиме с заниженной уставкой SetLimUsyn " + match[0]
    elif f.find("диапаз") > -1:
        result += "проверка диапазонов регулирования "
        if f.find("авт") > -1:
            result += "в автоматическом режиме"
        elif f.find("руч") > -1:
            result += "в ручном режиме"
    elif f.find("асим") > -1:
        result += "проверка защиты от асимметрии фазных токов"
    elif f.find("ug-ubar") > -1:
        result += "проверка подгонки напряжения генератора к напряжению сети"
    return result

def stability(f):
    regulator = f[:4].upper()

    ramp = [""]
    if re.search(r'[+-]\d+%?', f):
        ramp = re.search(r'[+-]\d+%?', f)
    
    time = [""]
    if re.search(r'\d+с', f):
        time = re.search(r'\d+с', f)
    
    mode = " в автоматическом режиме"
    if f.find("руч") > -1:
        mode = " в ручном режиме"
    
    return regulator + ", проверка устойчивости, толчек " + ramp[0] + " " + time[0].replace("с", " секунд") + mode

def run(files, function, root):
    result = {}
    for f in files:
        if f[-3:] != "png":
            continue
        result[root + "/" + f] = function(f)
    return result

In [3]:
def genPicturesList():
    result = {}
    for root, dirs, files in os.walk('.'):
        if root.find("возбуждение") > -1:
            result.update(run(files, excitation, root))
    for root, dirs, files in os.walk('.'):
        if root.find("устойчивость") > -1:
            result.update(run(files, stability, root))
    for root, dirs, files in os.walk('.'):
        if root.find("режимные") > -1:
            result.update(run(files, mode, root))
    for root, dirs, files in os.walk('.'):
        if root.find("переводы") > -1:
            result.update(run(files, transfers, root))

    return result

In [4]:
for i, k in enumerate(genPicturesList().keys()):
    print(i+1, genPicturesList()[k], sep='\t')

1	АРВ1, опыт начального возбуждения в автоматическом режиме
2	АРВ1, опыт начального возбуждения в ручном режиме
3	АРВ1, опыт гашения поля в автоматическом режиме
4	АРВ1, опыт гашения поля отключением АГП
5	АРВ1, опыт гашения поля в ручном режиме
6	АРВ2, опыт начального возбуждения в автоматическом режиме
7	АРВ2, опыт начального возбуждения в ручном режиме
8	АРВ2, опыт гашения поля в автоматическом режиме
9	АРВ2, опыт гашения поля отключением АГП
10	АРВ2, опыт гашения поля в ручном режиме
11	АРВ1, проверка устойчивости, толчек +3% 5 секунд в ручном режиме
12	АРВ1, проверка устойчивости, толчек -3% 5 секунд в ручном режиме
13	АРВ1, проверка устойчивости, толчек -5% 5 секунд в автоматическом режиме
14	АРВ2, проверка устойчивости, толчек +3% 5 секунд в ручном режиме
15	АРВ2, проверка устойчивости, толчек +5% 5 секунд в автоматическом режиме
16	АРВ2, проверка устойчивости, толчек -3% 5 секунд в ручном режиме
17	АРВ2, проверка устойчивости, толчек -5% 5 секунд в автоматическом режиме
18	АРВ1

In [5]:
document = Document()

for section in document.sections:
    section.top_margin = Cm(1)
    section.bottom_margin = Cm(1)
    section.left_margin = Cm(1.2)
    section.right_margin = Cm(1.2)

style = document.styles['Normal']
font = style.font
font.name = 'Times New Roman'
font.size = Pt(10)

p = document.add_paragraph('Осциллограммы опытов ТГ2 на холстом ходу')
document.add_page_break()
p.style = document.styles['Normal']
p.alignment = WD_ALIGN_PARAGRAPH.CENTER

pic_list = []
result = genPicturesList()
for i, fileName in enumerate(result.keys()):
    pic = document.add_picture(fileName, width=Cm(19))
    pic.style = document.styles['Normal']
    pic.alignment = WD_ALIGN_PARAGRAPH.CENTER
    pic_list.append(pic)
    
    document.add_heading("Рисунок 1.{}. ".format(i+1) + result[fileName] + ".", level=3)
    document.add_page_break()
    #r.add_break()

document.save('хх-тг2.docx')